In [1]:
# %run ./etl_trusted_features.ipynb

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/15 07:19:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/15 07:19:45 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
%pip install tensorflow --quiet
%pip install tensorflow_gnn --quiet
%pip install tensorflow_io --quiet

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathos 0.3.0 requires dill>=0.3.6, but you have dill 0.3.1.1 which is incompatible.
multiprocess 0.70.14 requires dill>=0.3.6, but you have dill 0.3.1.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow import keras
import tensorflow_gnn as tfgnn
from tensorflow_gnn import runner
from tensorflow.keras import layers
import matplotlib.pyplot as plt

2023-03-15 07:21:19.976067: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
def create_graph_tensor(df):
    graph_tensor = tfgnn.GraphTensor.from_pieces(
    context_spec = tfgnn.ContextSpec.from_field_specs(
        features_spec ={
            "malicious": np.array(df['malicious'],
                                dtype='string').reshape(len(df),1)
        }),
    node_sets = {
        "actorID": tfgnn.NodeSet.from_fields(
            sizes = [len(df)],
            features ={
                'parent_image_path': np.array(df['parent_image_path'],
                                dtype='string').reshape(len(df),1)
            }),
        "objectID": tfgnn.NodeSet.from_fields(
            sizes = [len(df)],
            features ={
                'base_address': np.array(df['base_address'],
                                 dtype='string').reshape(len(df),1),
                'file_path': np.array(df['file_path'],
                                   dtype='string').reshape(len(df),1),
                'image_path': np.array(df['image_path'],
                                   dtype='string').reshape(len(df),1),
            })
    },
    edge_sets ={
        "id": tfgnn.EdgeSet.from_fields(
            sizes = [len(df)],
            features = {
                'id': np.array(df['id'],
                                      dtype='string').reshape(len(df),1)},
            adjacency = tfgnn.Adjacency.from_indices(
                source = ("actorID", np.array(df['actorID'], dtype='string')),
                target = ("objectID", np.array(df['objectID'], dtype='string')))),
  })

    return graph_tensor

In [4]:
graph_schema = tfgnn.read_schema("./graph_schema.pbtxt")
gtspec = tfgnn.create_graph_spec_from_schema_pb(graph_schema)

In [5]:
gtspec

GraphTensorSpec({'context': ContextSpec({'features': {'malicious': RaggedTensorSpec(TensorShape([1, None]), tf.string, 1, tf.int32)}, 'sizes': TensorSpec(shape=(1,), dtype=tf.int32, name=None)}, TensorShape([]), tf.int32, None), 'node_sets': {'objectID': NodeSetSpec({'features': {'file_path': RaggedTensorSpec(TensorShape([None, None]), tf.string, 1, tf.int32), 'base_address': RaggedTensorSpec(TensorShape([None, None]), tf.string, 1, tf.int32), 'image_path': RaggedTensorSpec(TensorShape([None, None]), tf.string, 1, tf.int32)}, 'sizes': TensorSpec(shape=(1,), dtype=tf.int32, name=None)}, TensorShape([]), tf.int32, None), 'actorID': NodeSetSpec({'features': {'parent_image_path': RaggedTensorSpec(TensorShape([None, None]), tf.string, 1, tf.int32)}, 'sizes': TensorSpec(shape=(1,), dtype=tf.int32, name=None)}, TensorShape([]), tf.int32, None)}, 'edge_sets': {'id': EdgeSetSpec({'features': {'event_minute': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'event_hour': TensorSpec(shape=(N

In [33]:
gt = tfgnn.random_graph_tensor(gtspec)

2023-03-15 07:39:38.920187: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-15 07:39:38.920236: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-16-10-147.us-west-2.compute.internal): /proc/driver/nvidia/version does not exist
2023-03-15 07:39:38.921158: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
train_ds_provider = runner.TFRecordDatasetProvider(file_pattern="s3://sapient-bucket-trusted/prod/graph/first_events/")

In [7]:
valid_ds_provider = runner.TFRecordDatasetProvider(file_pattern="s3://sapient-bucket-trusted/prod/graph/first_events/")

In [8]:
graph_tensor_spec = tfgnn.create_graph_spec_from_schema_pb(graph_schema)
dataset = dataset.map(
    lambda serialized: tfgnn.parse_single_example(graph_tensor_spec, serialized))

for i, graph in enumerate(dataset.take(3)):
  print(f"Input {i}: {graph}")

NameError: name 'dataset' is not defined

In [30]:
def getFirstEventFiles():
    s3 = boto3.client('s3')
    response = s3.list_objects_v2(
        Bucket = 'sapient-bucket-trusted',
        Prefix = f'prod/')
    all_files = []
    for content in response.get('Contents', []):
        all_files.append(content['Key'])
        # print(content['Key'])
    files = [f"sapient-bucket-trusted/" + f for f in all_files if 'parquet' in f]

    return files

In [32]:
first_event_files = getFirstEventFiles()

In [ ]:
def map_fn(file_location):
    return tfio.IODataset.from_parquet(file_location, columns=columns)

In [ ]:
train_files = [ s3bucket + "/path/file_{}.parquet".format(i) for i in range(train_size) ]

In [ ]:
train_size = 640
batch_size = 32

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_files, train_labels))

In [ ]:
files = [ s3bucket + "/path/file_{}.parquet".format(i) for i in range(5) ]
ds = tf.data.Dataset.from_tensor_slices(files).map(map_fn)